# Challenges local development

This notebook was used for locally developing Python code for the challenges. It also provides some visualizations.

## Setup

In [ ]:
# yolo
import warnings

warnings.simplefilter("ignore")

## Installs

In [ ]:
#%pip install --upgrade 'snowflake-snowpark-python[pandas]' pyyaml numpy pandas scikit-learn matplotlib seaborn prophet mlflow thefuzz ipympl

## Imports

In [ ]:
import yaml
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from random import random, randint
from thefuzz import fuzz
from prophet import Prophet
from datetime import datetime
from sklearn.cluster import KMeans
from prophet.serialize import model_to_json, model_from_json
from snowflake.snowpark import Session
from sklearn.decomposition import PCA

from thefuzz.process import extractOne as match_str

## Connect to Snowpark in Python

In [ ]:
# replace as needed
PROFILE_PATH = "/root/.dbt/profiles.yml"
PROFILE_NAME = "snowflake"
PROFILE_OUTPUT = "dev"

# read in dbt profile
with open(PROFILE_PATH, "r") as f:
    profiles = yaml.safe_load(f)
    profile = profiles[PROFILE_NAME]["outputs"][PROFILE_OUTPUT]

# build connection parameters from profile
conn_params = {
    "account": profile["account"],
    "user": profile["user"],
    "role": profile["role"],
    "warehouse": profile["warehouse"],
    "database": profile["database"],
    "schema": profile["schema"],
    "authenticator": profile["authenticator"],
}
conn_params

In [ ]:
schema = conn_params["schema"]

In [ ]:
from snowflake.snowpark import Session

s = Session.builder.configs(conn_params).create()
s

In [ ]:
s.sql("select current_warehouse(), current_database(), current_schema()").collect()

## Challenge 1: describe the data

In [ ]:
orders = s.table(f"{schema}.orders")
orders.show(5)

In [ ]:
described = orders.describe()
described.show()

## Challenge 2: pivot the data

Resorting to Pandas.

In [ ]:
orders = s.table(f"{schema}.orders").sample(frac=0.2).to_pandas()
orders.head(5)

In [ ]:
order_items = s.table(f"{schema}.stg_order_items").sample(frac=0.2).to_pandas()
order_items.head()

In [ ]:
products = s.table(f"{schema}.stg_products").to_pandas()
products.head()

In [ ]:
product_ids = sorted(list(set(products["product_id".upper()].unique())))
product_ids

In [ ]:
order_item_product_subtotals = (
    order_items.merge(products, on="product_id".upper())
    .groupby(
        ["order_id".upper(), "product_id".upper()],
        as_index=False,
    )
    .agg(SUBTOTAL=("product_price".upper(), "sum"))
    .reset_index()
    .pivot(
        index="order_id".upper(),
        columns="product_id".upper(),
        values="subtotal".upper(),
    )
    .reset_index()
)
order_item_product_subtotals.head()

In [ ]:
renames = {product_id: f"subtotal_{product_id}".upper() for product_id in product_ids}
order_item_product_subtotals = order_item_product_subtotals.rename(columns=renames)
order_item_product_subtotals.head()

In [ ]:
order_item_product_subtotals = order_item_product_subtotals.fillna(0)
order_item_product_subtotals.head()

In [ ]:
orders_with_subtotals = orders.merge(
    order_item_product_subtotals, on="order_id".upper()
)
orders_with_subtotals.head()

## Challenge 3: flag fuzzy duplicates

In [ ]:
customers = s.table(f"{schema}.customers").to_pandas()
customers.head()

In [ ]:
drop_cols = ["customer_id", "customer_order_index", "is_first_order"]
drop_cols.extend([col for col in customers.columns if col != "customer_name".upper()])
drop_cols = [col.upper() for col in drop_cols]

fuzzed = orders.merge(customers, on="customer_id".upper()).drop(drop_cols, axis=1)
fuzzed.tail()

In [ ]:
names = fuzzed["customer_name".upper()]
names.head()

In [ ]:
def fuzz_name(name):

    fuzz_name = ""
    names = name.split(" ")

    for name in names:
        if random() < 0.5:
            # employee or AI is decisive
            if random() < 0.5:
                # and loves all caps
                name = name.upper()
            else:
                # or all lowercase
                name = name.lower()

        if random() < 0.2:
            # AI dropped the first or last letter probably :/
            if random() < 0.5:
                # first letter dropped, whoops
                name = name[1:]
            else:
                # last letter dropped, whoops
                name = name[:-1]

        if random() < 0.1:
            # a solar flare hit the datacenter in all regions,
            # no multi-region resiliency could have saved it :(
            for char in name:
                if random() < 0.3:
                    name = name.replace(char, chr(ord(char) + randint(-5, 5)))

        fuzz_name += name + " "

    return fuzz_name.strip()

In [ ]:
fuzzed_names = [fuzz_name(name) for name in names]
fuzzed_names[0:10]

In [ ]:
fuzzed["customer_name".upper()] = fuzzed_names
fuzzed.head()

In [ ]:
customer_names = sorted(list(set(customers["customer_name".upper()].unique())))
customer_names[0:10]

In [ ]:
# this takes a very long time to run
if False:
    unfuzzed = fuzzed
    unfuzzed["customer_name_unfuzeed".upper()] = fuzzed["customer_name".upper()].apply(
        lambda x: match_str(x, customer_names)[0]
    )
    unfuzzed.head()

## Challenge 4: cluster customers by their order history

In [ ]:
df = orders_with_subtotals
df.head()

In [ ]:
X = df.select_dtypes(include=["number"]).values
X

Detour: motivation!

In [ ]:
n_components = 3
pca = PCA(n_components=n_components)
pca

In [ ]:
pca = pca.fit(X)
pca

In [ ]:
X_pca = pca.transform(X)
X_pca.shape

In [ ]:
%matplotlib ipympl

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection="3d")

ax.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    X_pca[:, 2],
    c=df["subtotal".upper()].values,
)

In [ ]:
model = KMeans(n_clusters=5)
model

In [ ]:
model = model.fit(X)
model

In [ ]:
cluster_labels = model.predict(X)
cluster_labels

In [ ]:
%matplotlib ipympl

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection="3d")

ax.scatter(
    X_pca[:, 0],
    X_pca[:, 1],
    X_pca[:, 2],
    c=cluster_labels,
)

In [ ]:
temp = pd.DataFrame(data=cluster_labels, columns=["cluster_label"])
temp

In [ ]:
orders_with_subtotals_and_clusters = orders_with_subtotals.merge(
    temp, left_index=True, right_index=True
)
orders_with_subtotals_and_clusters.head()

## Challenge 5: predict revenue by location

In [ ]:
revenue = s.table(f"{schema}.revenue_weekly_by_location").to_pandas()
revenue.head()

In [ ]:
renames = {
    "date_week".upper(): "ds",
    "location_name".upper(): "location",
    "revenue".upper(): "y",
}
revenue = revenue.rename(columns=renames)
revenue.head()

In [ ]:
locations = sorted(list(revenue["location"].unique()))
locations

In [ ]:
models = {
    location: Prophet().fit(revenue[revenue["location"] == location])
    for location in locations
}
models

In [ ]:
future = models[locations[0]].make_future_dataframe(periods=52 * 3, freq="W")
future.tail()

In [ ]:
forecasts = {location: models[location].predict(future) for location in locations}
forecasts

In [ ]:
for location in locations:
    models[location].plot(forecasts[location])
    plt.title(location)

Now using the dbt models

In [ ]:
models = s.table(f"{schema}.forecast_train_py").to_pandas()
models.tail()

In [ ]:
locations = sorted(list(models["location"].unique()))
locations

In [ ]:
most_recent_trained_at = models["trained_at"].max()
most_recent_trained_at

In [ ]:
models = models[models["trained_at"] == most_recent_trained_at]
models

In [ ]:
models = {
    location: model_from_json(models[models["location"] == location]["model"].iloc[0])
    for location in locations
}
models

In [ ]:
forecasts = s.table(f"{schema}.forecast_score_py").to_pandas()
forecasts.tail()

In [ ]:
# hack -- datetime in Snowpark/Pandas funkiness
forecasts["ds"] /= 1e6
forecasts["ds"] = pd.to_datetime(forecasts["ds"], unit="s")
forecasts.info()

In [ ]:
for location in locations:
    models[location].plot(forecasts[forecasts["location"] == location])
    plt.title(location)